In [64]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains as ac
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, NoSuchElementException
import json
import time
import re

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

In [63]:
class findJobsOnLinkedIn:

    # 參數初始化
    def __init__(self, data):
        self.email = data["email"]
        self.password = data["password"]
        self.keywords = data["keywords"]
        self.location = data["location"]

    # 進入 LinkedIn 網頁並登入
    def login_LinkedIn(self):
        driver.get("https://www.linkedin.com/")
        
        login_email = driver.find_element(By.CSS_SELECTOR, "#session_key")
        login_email.clear()
        login_email.send_keys(self.email)
        login_password = driver.find_element(By.CSS_SELECTOR, "#session_password")
        login_password.clear()
        login_password.send_keys(self.password)
        login_password.send_keys(Keys.RETURN)
    
    # 找尋對應關鍵字職缺
    def job_search(self):
        # 進入職缺頁面
        jobs_link = driver.find_element(By.LINK_TEXT, "職缺")
        jobs_link.click()
        time.sleep(2)
        
        # 輸入關鍵字及地點
        inputKeywords = driver.find_element(By.XPATH,'//input[starts-with(@id,"jobs-search-box-keyword")]')
        inputKeywords.clear()
        inputKeywords.send_keys(self.keywords)
        time.sleep(1)
        inputLocation = driver.find_element(By.XPATH,'//input[starts-with(@id,"jobs-search-box-location")]')
        inputLocation.clear()
        inputLocation.send_keys(self.location)
        time.sleep(1)
        inputLocation.send_keys(Keys.RETURN)
        time.sleep(1)
        
    def filter(self):
        easyApply_button = driver.find_element(By.XPATH, "//button[@aria-label='篩選條件：一鍵應徵']")
        easyApply_button.click()
    
    # 從所有頁面抓取所有職缺結果
    def find_offers(self):
        # 取得職缺總數
        total_results = driver.find_element(By.CSS_SELECTOR, "div.jobs-search-results-list__subtitle")
        total_results_int = int(total_results.text.split(" ", 1)[0].replace(",",""))
        print(total_results_int)
        
        time.sleep(2)
        
        # 取得第一頁職缺數量
        current_page = driver.current_url
        results = driver.find_elements(By.CLASS_NAME, "ember-view.jobs-search-results__list-item.occludable-update.p0.relative.scaffold-layout__list-item")
                                       
        for result in results:
            hover = ac(driver).move_to_element(result)
            hover.perform()
            titles = result.find_element(By.CLASS_NAME,"disabled.ember-view.job-card-container__link.job-card-list__title")
            for title in titles:
                #點擊一鍵應徵，如果已應徵則跳過
                try:
                    in_apply = driver.find_element(By.CSS_SELECTOR,"div.jobs-apply-button--top-card")
                    in_apply.click()
                    time.sleep(1)

                    discard = self.driver.find_element(By.XPATH, "//button[@data-test-modal-close-btn]")
                    discard.send_keys(Keys.RETURN)

                    time.sleep(1)

                    discard_confirm = self.driver.find_element(By.XPATH,"//button[@data-test-dialog-secondary-btn]")
                    discard_confirm.send_keys(Keys.RETURN)

                    time.sleep(1)               
                except NoSuchElementException:
                    print('已應徵過此職缺')
                    pass
                time.sleep(1)   
            
        # 如果職缺數量大於一頁，載入其他頁面並取得結果
        if total_results_int > 24:
            
            time.sleep(2)
            
            # 找到最後一頁，以此為總頁數來建立每一頁的 url
            find_pages = driver.find_elements(By.CLASS_NAME,"artdeco-pagination__indicator.artdeco-pagination__indicator--number.ember-view")
            total_pages = find_pages[len(find_pages)-1].text
            total_pages_int = int(re.sub(r"[^\d.]", "", total_pages))
            
            page = str(total_pages_int)
            get_last_page = driver.find_element(By.XPATH,f'//button[@aria-label="第 {page} 頁"]')
            get_last_page.send_keys(Keys.RETURN)
            time.sleep(2)
            
            last_page = driver.current_url
            total_jobs = int(last_page.split('start=',1)[1]) #網址從 start 切割，取第二段
            
            # 走訪所有頁面
            for page_number in range(25,total_jobs+25,25):
                driver.get(current_page+'&start='+str(page_number))
                time.sleep(2) 
                results_ext = driver.find_elements(By.CLASS_NAME, "ember-view.jobs-search-results__list-item.occludable-update.p0.relative.scaffold-layout__list-item")
                # apply jobs
                for result_ext in results_ext:
                    hover_ext = ac(driver).move_to_element(result_ext)
                    hover_ext.perform()
                    titles_ext = result_ext.find_elements(By.CLASS_NAME,"disabled.ember-view.job-card-container__link.job-card-list__title")
                    for title_ext in titles_ext:
                        self.application_submit(title_ext)
        else:
            self.close_session()
    
    def application_submit(self, job_ad):
        print("您應徵職缺 :", job_ad.text)
        job_ad.click()
        time.sleep(2)
        
        #點擊一鍵應徵，如果已應徵則跳過
        try:
            in_apply = driver.find_element(By.CSS_SELECTOR,"div.jobs-apply-button--top-card")
            in_apply.click()
            time.sleep(1)

            discard = self.driver.find_element(By.XPATH, "//button[@data-test-modal-close-btn]")
            discard.send_keys(Keys.RETURN)

            time.sleep(1)

            discard_confirm = self.driver.find_element(By.XPATH,"//button[@data-test-dialog-secondary-btn]")
            discard_confirm.send_keys(Keys.RETURN)

            time.sleep(1)               
        except NoSuchElementException:
            print('已應徵過此職缺')
            pass
        time.sleep(1)     
        
    def close_session(self):
        
        print('執行結束')
        driver.close()
        
    def apply_jobs(self):
        self.login_LinkedIn()
        time.sleep(3)
        self.job_search()
        time.sleep(2)
        self.filter()
        time.sleep(2)
        self.find_offers()
        time.sleep(2)
        self.close_session()


In [65]:
if __name__ == "__main__":
    with open("config.json","r",encoding="utf-8") as file:
        data = json.load(file)
        
    bot = findJobsOnLinkedIn(data)
    bot.apply_jobs()

337


TypeError: 'WebElement' object is not iterable